## Set things up

In [1]:
!pip3 install --quiet -U transformers datasets

In [5]:
!pip3 install --quiet -U captum pandas matplotlib seaborn

In [2]:
# relevant stuff repeated from above so you can run it from here onwards if you happen to have the model trained

from transformers import AutoTokenizer, AutoModelForSequenceClassification

MODEL_NAME = 'all'

tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-cased") #("TurkuNLP/bert-base-finnish-cased-v1")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

In [16]:
# test tokenization

text = "This is a text meant to visualize how texts are tokenized by the bert-base-cased model."
inp=tokenizer(text) # , return_tensors="pt"
#print(inp)
all_tokens = tokenizer.convert_ids_to_tokens(inp["input_ids"])
print(text)
print(" ".join(all_tokens))

print(inp)
print(inp["input_ids"])
print(inp["token_type_ids"])


This is a text meant to visualize how texts are tokenized by the bert-base-cased model.
[CLS] This is a text meant to visual ##ize how texts are token ##ized by the be ##rt - base - case ##d model . [SEP]
{'input_ids': [101, 1188, 1110, 170, 3087, 2318, 1106, 5173, 3708, 1293, 6685, 1132, 22559, 2200, 1118, 1103, 1129, 3740, 118, 2259, 118, 1692, 1181, 2235, 119, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}
[101, 1188, 1110, 170, 3087, 2318, 1106, 5173, 3708, 1293, 6685, 1132, 22559, 2200, 1118, 1103, 1129, 3740, 118, 2259, 118, 1692, 1181, 2235, 119, 102]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [ ]:
label = ['not-toxic', 'toxic']

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')
%cd /content/drive/My Drive/Colab Notebooks/dippa

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
/content/drive/My Drive/Colab Notebooks/dippa


In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME)

In [ ]:
import datasets

dataset = datasets.load_dataset("csv", data_files="all_comparison.csv", sep=",", index_col=[0]) # all_reannotated.tsv, sep="t"
dataset = dataset["train"]
dataset = dataset.remove_columns(["__index_level_0__"])

Generating train split: 0 examples [00:00, ? examples/s]

## Sorting stuff

In [ ]:
# TODO sort the things to different datasets
# https://huggingface.co/docs/datasets/process#select-and-filter

toxic = dataset.filter(lambda example: example["pred_label"] == 1) # toxic and labelled as such  example["true_label"] == 1
most_toxic = toxic.sort("probability", reverse=True) # sort really toxic by highest toxicity
#most_toxic = most_toxic.remove_columns(["true_label", "pred_label"])

probabilities_sorted = dataset.sort("probability", reverse=True)


false_negative = dataset.filter(lambda example: example["true_label"] == 1 and example["pred_label"] == 0) # toxic but labelled as not
false_negative = false_negative.sort("probability") #-> labelled as not-toxic with high probability

false_positive = dataset.filter(lambda example: example["true_label"] == 0 and example["pred_label"] == 1) # not-toxic but labelled as toxic
false_positive = false_positive.sort("probability") #-> labelled as toxic with high probability but is actually not

middle_ground = dataset.filter(lambda example: example["probability"] >= 0.5 and example["probability"] <= 0.80)
middle_ground = middle_ground.sort("probability")

In [ ]:
probabilities_sorted.to_csv("probs_sorted.csv")
middle_ground.to_csv("middle_ground.csv")

Creating CSV from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Creating CSV from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

5192

In [ ]:
most_toxic.to_csv("most_toxic.csv")

Creating CSV from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

99553

In [ ]:
false_negative.to_csv("false_negative.csv")
false_positive.to_csv("false_positive.csv")

Creating CSV from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Creating CSV from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

27566

## Explainability

In [ ]:
def predict_answer(example):
    #print(example)
    text=example["text"]
    tokenized = tokenizer(text, truncation=True, return_tensors='pt')
    #print(tokenized)
    pred = model(**tokenized)
    pred_idx = pred.logits.detach().numpy().argmax()
    return label[pred_idx]

In [ ]:
example_sentences = dataset
example_sentences

Dataset({
    features: ['text', 'true_label', 'pred_label', 'probability'],
    num_rows: 600
})

In [ ]:
for i in range(len(example_sentences[:5])):
    print(example_sentences[i]["text"], '->', predict_answer(example_sentences[i]))

musta se taas on ihan sikahuono!! -> not-toxic
Ja audii, mersuu, bemarii.... -> not-toxic
idiootti. Minkäs sille mahtaa. Käypä katsomassa kelit joskus itsekin, lennot ei paljoa maksa. -> toxic
Elokuva halosesta tehtiin verovaroilla ?  Halonen käyttäytyi kuten muutkin kommunistijohtajat.  Paskasakkia. -> toxic


In [ ]:
from captum.attr import visualization as viz
from captum.attr import IntegratedGradients, LayerConductance, LayerIntegratedGradients
from captum.attr import configure_interpretable_embedding_layer, remove_interpretable_embedding_layer

In [ ]:
#Tells the model that it is in evaluation mode, and zeroes out the gradients

model.eval()
model.zero_grad()

In [ ]:
# Forward on the model -> data in, prediction out, nothing fancy really
def predict(inputs, token_type_ids, attention_mask):
    pred=model(inputs, token_type_ids=token_type_ids, attention_mask=attention_mask)
    return pred.logits #return the output of the classification layer

In [ ]:
# Given input text, construct a pair of (text input, blank reference input as long as the text itself)
def construct_input_ref(text):
    inp=tokenizer(text, return_tensors="pt", truncation=True, max_length=100)
    #let's have our blank be all padding
    #but how long is the text?
    #one way to do this:
    inp_len = tokenizer(text, add_special_tokens=False, truncation=True, max_length=98)["input_ids"]
    ref=tokenizer(text=" ".join(["[PAD]"]*len(inp_len)),return_tensors="pt", truncation=True)
    return (inp["input_ids"],inp["token_type_ids"],inp["attention_mask"]),\
    (ref["input_ids"],ref["token_type_ids"],ref["attention_mask"])

In [ ]:
# Let's try it!

text=dataset[0]["text"]

#input:
inp, ref = construct_input_ref(text)
all_tokens = tokenizer.convert_ids_to_tokens(inp[0][0])
ref_tokens = tokenizer.convert_ids_to_tokens(ref[0][0])
print(all_tokens)
print(ref_tokens)

['[CLS]', 'musta', 'se', 'taas', 'on', 'ihan', 'sika', '##huon', '##o', '!', '!', '[SEP]']
['[CLS]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[SEP]']


In [ ]:
# Yay, now we finally made it to the attribution part
lig = LayerIntegratedGradients(predict, model.bert.embeddings) #attribute the output wrt to embeddings

In [ ]:
# inputs: inputs
# baselines: the blank baseline
# target: which of the two classes in the output (pos/neg) to run the prediction against?
attrs, delta = lig.attribute(inputs=inp,
                                  baselines=ref,
                                  return_convergence_delta=True,target=1)
print("attrs shape",attrs.shape)

attrs shape torch.Size([1, 12, 768])


In [ ]:
import torch

In [ ]:
def summarize_attributions(attributions):
    attributions = attributions.sum(dim=-1).squeeze(0)
    attributions = attributions / torch.linalg.vector_norm(attributions,dim=0)
    return attributions

attrs_sum = summarize_attributions(attrs)
print("attrs_sum shape",attrs_sum.shape)

attrs_sum shape torch.Size([12])


In [ ]:
print(attrs_sum)
print(tokenizer.convert_ids_to_tokens(inp[0][0]))

for a,t in zip(attrs_sum,tokenizer.convert_ids_to_tokens(inp[0][0])):
    print(float(a),t)

tensor([ 0.0000,  0.0740,  0.1327,  0.1986, -0.0913,  0.1900, -0.2990, -0.8958,
         0.0196,  0.0230, -0.0134,  0.0000])
['[CLS]', 'musta', 'se', 'taas', 'on', 'ihan', 'sika', '##huon', '##o', '!', '!', '[SEP]']
0.0 [CLS]
0.07399281859397888 musta
0.13266943395137787 se
0.1986301988363266 taas
-0.09133460372686386 on
0.19004906713962555 ihan
-0.29898595809936523 sika
-0.89583820104599 ##huon
0.019585689529776573 ##o
0.02295035682618618 !
-0.013428030535578728 !
0.0 [SEP]


In [ ]:
import captum
from IPython.core.display import HTML, display
x=captum.attr.visualization.format_word_importances(tokenizer.convert_ids_to_tokens(inp[0][0]),attrs_sum)
HTML(x)

In [ ]:
import torch
def predict_and_explain(model,text):
    model.zero_grad() #to be safe perhaps it's not needed
    inp,ref = construct_input_ref(text)
    all_tokens = tokenizer.convert_ids_to_tokens(inp[0][0])

    lig = LayerIntegratedGradients(predict, model.bert.embeddings)
    prediction=predict(*inp)[0]
    prediction_cls=int(torch.argmax(prediction))
    probs = torch.nn.functional.softmax(prediction)
    probs_float = torch.argmax(probs).detach().numpy() # what the hell goes wrong here with the argmax
    print("Prediction:", ("not-toxic","toxic")[prediction_cls], "Weights:",prediction.tolist()) # "Probability:", f'{probs_float:.{5}f}'
    for target,classname in enumerate(("not-toxic","toxic")):

        attrs, delta = lig.attribute(inputs=inp,
                                  baselines=ref,
                                  return_convergence_delta=True,target=target)
        attrs_sum = summarize_attributions(attrs)

        x=captum.attr.visualization.format_word_importances(all_tokens,attrs_sum)
        print(text)
        print("ATTRIBUTION WITH RESPECT TO",classname)
        display(HTML(x))
        print()

## Visualization!

In [13]:
from IPython.display import HTML, display

def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))
get_ipython().events.register('pre_run_cell', set_css)

In [ ]:
# take the middle ground texts

middle = dataset.filter(lambda example: example["probability"] >= 0.5 and example["probability"] <= 0.60)
middle = middle.sort("probability")

In [ ]:
for e in middle["text"][1:]:
  predict_and_explain(model, e)

Prediction: toxic Weights: [0.3194977939128876, 0.3732896149158478]
kuitenkaan mikään feministi oo..
ATTRIBUTION WITH RESPECT TO not-toxic



kuitenkaan mikään feministi oo..
ATTRIBUTION WITH RESPECT TO toxic



Prediction: toxic Weights: [0.18053768575191498, 0.26253074407577515]
Asiasta: Blondit ja äly. Kaikki blondit eivät ole kauniita mielestäni. Vain älykkäät blondit ovat kauniita, äly näkyy silmissä. -Tyhmä "Gro magno" ei ole mielestäni koskaan kaunis.
ATTRIBUTION WITH RESPECT TO not-toxic



Asiasta: Blondit ja äly. Kaikki blondit eivät ole kauniita mielestäni. Vain älykkäät blondit ovat kauniita, äly näkyy silmissä. -Tyhmä "Gro magno" ei ole mielestäni koskaan kaunis.
ATTRIBUTION WITH RESPECT TO toxic



Prediction: not-toxic Weights: [0.30260327458381653, 0.055026646703481674]
Pillu, perse, kyrpä/mulkku, tissit/buubsit, pano tms. Kuuluvat sujuvasti sanavalikoimaani myös miesten kanssa jutellessa. Mitä sitä turhia keksimään uusia sanoja.
ATTRIBUTION WITH RESPECT TO not-toxic



Pillu, perse, kyrpä/mulkku, tissit/buubsit, pano tms. Kuuluvat sujuvasti sanavalikoimaani myös miesten kanssa jutellessa. Mitä sitä turhia keksimään uusia sanoja.
ATTRIBUTION WITH RESPECT TO toxic


In [ ]:
for e in ["PRESIDENTIN PITÄISI YHDISTÄÄ KANSAA", "Presidentin pitäisi yhdistää kansaa"]:
  predict_and_explain(model, e)

/tmp/ipython-input-87-4225573040.py:10: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  probs = torch.nn.functional.softmax(prediction)


Prediction: not-toxic Probability: 0.0 Weights: [3.363703727722168, -3.5651063919067383]
PRESIDENTIN PITÄISI YHDISTÄÄ KANSAA
ATTRIBUTION WITH RESPECT TO not-toxic



PRESIDENTIN PITÄISI YHDISTÄÄ KANSAA
ATTRIBUTION WITH RESPECT TO toxic



Prediction: not-toxic Probability: 0.0 Weights: [3.2067863941192627, -3.4443042278289795]
Presidentin pitäisi yhdistää kansaa
ATTRIBUTION WITH RESPECT TO not-toxic



Presidentin pitäisi yhdistää kansaa
ATTRIBUTION WITH RESPECT TO toxic


In [ ]:
predict_and_explain(model, "Tää on aika hyvä:  Mrs Weasley: Not my daughter, you bitch!")

/tmp/ipython-input-94-15130911.py:10: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  probs = torch.nn.functional.softmax(prediction)


Prediction: toxic Weights: [-3.964351177215576, 4.56817102432251]
Tää on aika hyvä:  Mrs Weasley: Not my daughter, you bitch!
ATTRIBUTION WITH RESPECT TO not-toxic



Tää on aika hyvä:  Mrs Weasley: Not my daughter, you bitch!
ATTRIBUTION WITH RESPECT TO toxic


In [ ]:
# take the ones I chose to showcase or example["text"].startswith("Etelän Vetelillä") or example["text"].startswith("Ja instassa")
chosen = dataset.filter(lambda example: example["text"].startswith("on tää typerä") or example["text"].startswith("Sairasta") or example["text"].startswith("Tää on aika hyvä"))

Filter:   0%|          | 0/600 [00:00<?, ? examples/s]

In [ ]:
print(chosen["text"])

Column(['on tää typerä koirahommakin mennyt monimutkaiseksi siis jollakin on tai mieluumin ei ole koiraa.Mun puolesta kaikki paitsi sokeitten koirat voi hyvinkin ampua.', 'Sairasta touhua. Joutas sanonko minne.... hyi helvetti....', 'Tää on aika hyvä:  Mrs Weasley: NOT MY DAUGHTER, YOU BITCH!'])


In [ ]:
for e in chosen["text"]:
  predict_and_explain(model, e)

Prediction: not-toxic Weights: [0.5817758440971375, -0.19104090332984924]
on tää typerä koirahommakin mennyt monimutkaiseksi siis jollakin on tai mieluumin ei ole koiraa.Mun puolesta kaikki paitsi sokeitten koirat voi hyvinkin ampua.
ATTRIBUTION WITH RESPECT TO not-toxic



on tää typerä koirahommakin mennyt monimutkaiseksi siis jollakin on tai mieluumin ei ole koiraa.Mun puolesta kaikki paitsi sokeitten koirat voi hyvinkin ampua.
ATTRIBUTION WITH RESPECT TO toxic



Prediction: toxic Weights: [-3.1568100452423096, 3.3085341453552246]
Sairasta touhua. Joutas sanonko minne.... hyi helvetti....
ATTRIBUTION WITH RESPECT TO not-toxic



Sairasta touhua. Joutas sanonko minne.... hyi helvetti....
ATTRIBUTION WITH RESPECT TO toxic



Prediction: toxic Weights: [-3.4934895038604736, 3.8016648292541504]
Tää on aika hyvä:  Mrs Weasley: NOT MY DAUGHTER, YOU BITCH!
ATTRIBUTION WITH RESPECT TO not-toxic



Tää on aika hyvä:  Mrs Weasley: NOT MY DAUGHTER, YOU BITCH!
ATTRIBUTION WITH RESPECT TO toxic


In [ ]:
print(dataset[5])
predict_and_explain(model, dataset[5]["text"])

{'text': 'vähä semmonen homppelipoika', 'true_label': 1, 'pred_label': 1, 'probability': 0.98989737033844}
Prediction: toxic Weights: [-2.217491388320923, 2.367309093475342]
vähä semmonen homppelipoika
ATTRIBUTION WITH RESPECT TO not-toxic



vähä semmonen homppelipoika
ATTRIBUTION WITH RESPECT TO toxic


In [ ]:
for e in most_toxic["text"][:10]:
  predict_and_explain(model, e)

Prediction: toxic Weights: [-3.9984967708587646, 4.582043170928955]
Ime sä kuule sykkivääs.. Mullo muskelit!
ATTRIBUTION WITH RESPECT TO not-toxic



Ime sä kuule sykkivääs.. Mullo muskelit!
ATTRIBUTION WITH RESPECT TO toxic



Prediction: toxic Weights: [-3.98814058303833, 4.583622932434082]
Haista Vittu  Fuck You.
ATTRIBUTION WITH RESPECT TO not-toxic



Haista Vittu  Fuck You.
ATTRIBUTION WITH RESPECT TO toxic



Prediction: toxic Weights: [-3.9801809787750244, 4.588482856750488]
Vit-tu, että osaakin olla ruma ämmä!!! :-):-):-)
ATTRIBUTION WITH RESPECT TO not-toxic


...


Sori jos mä vittuilin.  koska olet ateistikusipää.
ATTRIBUTION WITH RESPECT TO toxic



Prediction: toxic Weights: [-3.931119203567505, 4.527619361877441]
Susku on huora kuten sinäkin, ikääntynyt rusinoitunut huora. Nainen, joka nielee mällejä pitäisi mielestäni keittää elävältä.
ATTRIBUTION WITH RESPECT TO not-toxic



Susku on huora kuten sinäkin, ikääntynyt rusinoitunut huora. Nainen, joka nielee mällejä pitäisi mielestäni keittää elävältä.
ATTRIBUTION WITH RESPECT TO toxic



Prediction: toxic Weights: [-3.9413206577301025, 4.506113529205322]
Hintti hilluu, kun ei saa pilluu, taitaa olla homoo, tai muuten latvassa lahoo! Hintithän käteen vetää, kun ei pääse muuten kokeileen ketää!
ATTRIBUTION WITH RESPECT TO not-toxic



Hintti hilluu, kun ei saa pilluu, taitaa olla homoo, tai muuten latvassa lahoo! Hintithän käteen vetää, kun ei pääse muuten kokeileen ketää!
ATTRIBUTION WITH RESPECT TO toxic


In [ ]:
for e in false_negative["text"][:-10]:
  predict_and_explain(model, e)

Prediction: toxic Weights: [-1.9032360315322876, 2.162867784500122]
>  Tässä tunnustat, että olen oikeassa: hiilihydraatti ei ole sama asia kuin sokeri. Miksi ihmeessä sitten vielä kiellät tämän faktan ?  >  Niin sitä helposti "kyllästyy" kun tulee eteen tilanne, jossa on vaan pakko tunnustaa olevansa väärässä. Tee se nyt, niin päästään eteenpäin.  >  Vitut, kirjoitit hiilihydraatit = sokeri. Älä yritä perääntyä väitteistäsi, ääliö.  Nyt sitten lue oikein helvetillisen tarkasti tämän keskustelun aloitus. Siinä lukee mm. nämä faktat: japanilaiset eivät kärsineet lihavuudesta eikä sen liitännäissairauksista kun heidän ruokavalionsa koostui erittäin suurelta osin hiilihydraatista (jopa 67 E-%). Vasta myöhemmin kun heidän hiilihydraattikuormansa on vähentynyt ja rasvakuorma lisääntynyt, niin heidän diabetesluvut ovat alkaneet nousta. Miksei 67 E-% hiilihydraattia syövät ihmiset ole lihavia vaikka hiilihydraatti mielestäsi lihottaa ? Aivan, väittämäsi on täyttä hihhulipaskaa. Hiilihydraatit


>  Tässä tunnustat, että olen oikeassa: hiilihydraatti ei ole sama asia kuin sokeri. Miksi ihmeessä sitten vielä kiellät tämän faktan ?  >  Niin sitä helposti "kyllästyy" kun tulee eteen tilanne, jossa on vaan pakko tunnustaa olevansa väärässä. Tee se nyt, niin päästään eteenpäin.  >  Vitut, kirjoitit hiilihydraatit = sokeri. Älä yritä perääntyä väitteistäsi, ääliö.  Nyt sitten lue oikein helvetillisen tarkasti tämän keskustelun aloitus. Siinä lukee mm. nämä faktat: japanilaiset eivät kärsineet lihavuudesta eikä sen liitännäissairauksista kun heidän ruokavalionsa koostui erittäin suurelta osin hiilihydraatista (jopa 67 E-%). Vasta myöhemmin kun heidän hiilihydraattikuormansa on vähentynyt ja rasvakuorma lisääntynyt, niin heidän diabetesluvut ovat alkaneet nousta. Miksei 67 E-% hiilihydraattia syövät ihmiset ole lihavia vaikka hiilihydraatti mielestäsi lihottaa ? Aivan, väittämäsi on täyttä hihhulipaskaa. Hiilihydraatit eivät lihota. Ainoastaan ylimääräiset syödyt kalorit lihottaa.  > 


Prediction: not-toxic Weights: [0.581749677658081, -0.19101488590240479]
on tää typerä koirahommakin mennyt monimutkaiseksi siis jollakin on tai mieluumin ei ole koiraa.Mun puolesta kaikki paitsi sokeitten koirat voi hyvinkin ampua.
ATTRIBUTION WITH RESPECT TO not-toxic



on tää typerä koirahommakin mennyt monimutkaiseksi siis jollakin on tai mieluumin ei ole koiraa.Mun puolesta kaikki paitsi sokeitten koirat voi hyvinkin ampua.
ATTRIBUTION WITH RESPECT TO toxic



Prediction: not-toxic Weights: [2.738222599029541, -2.8199622631073]
PRESIDENTIN PITÄISI YHDISTÄÄ KANSAA  PRESIDENTTI VIRKAAN ASTUESSAAN LUOVUTTAA PUOLUEEN JÄSENKIRJANSA POIS. HÄNEN PITÄISI OLLA KANSAN YHDISTÄVÄ JA EDUSTAA KAIKKIA SUOMESSA TASAVEROISESTI. HÄN EI EDUSTA SITÄ RYHMÄÄ TAI PUOLUETTA MISTÄ KUTEN HAAVISTO VIHREISTÄ JA NIINISTÖ KOKOOMUS.  TARJA HALOSEN ONGELMA OLI ETTÄ EDUSTI VAIN DEMAREITA. HAAVISTO EI VOI EDUSTAA VAIN VIHREIDEN JA HOMOJEN ASIAA. MYÖS PERSUJEN, KESKUSTAN, DEMAREIDEN JA KRISTILLISTEN..MELKOINEN SAVOTTAA ODOTTAA.  JOS HÄN HALUAISI OLLA MYÖS HETEROIDEN PRESIDENTTI
ATTRIBUTION WITH RESPECT TO not-toxic


...


Lukekaa miten se armeija oikeasti käydään.  Muutama keikka on tosi tarina MIEHESTÄ joka uskalsi olla oma itsensä, myös armeijassa. VAROITUS! Ei lälläri pojille ( muka miehille???). lue eläkä lässytä. https://muutama-keikka.webnode.fi jos et jaksa lukea ensin oikeasta elämästä vaan haluaisit lukea äidin sylistä (valita, äidin sylistä ei kerrota mitään, mene neuvolaan), niin sieltä sivulta löytyy armeija "osia" erikseen kun kelaat alas päin. Hyvää lukuhetkeä toivottaa sinulle armeijan setä, ja paskat.
ATTRIBUTION WITH RESPECT TO toxic



Prediction: not-toxic Weights: [3.973619222640991, -4.399588108062744]
Eikös tämä Sammaljoki laske Kokemäenjokeen siinä Äetsä alapuolella, ja se Kokemäenjoki taas puolestaan laskee mereen?  " Turpaa esiintyy vain etelä- ja lounaisrannikon murtovesissä sekä niihin laskevissa joissa. ". Tämähän se taisi olla väitteeni?  En oletakaan, että tietäisit kaloista. Mutta miksi sinulle on niin tärkeää yrittää päteä asiassa, josta ei taida tietää mitään. Eihän ne faktat - veli hopea - jankuttamalla muuksi muutu. Eikä se muutu winkkarikaan, kusetusta mikä kusetusta.  Ymmärtäisit jo nousta siitä Sarviksen pulkasta ja olla kuin muutkin miehet, jooko.
ATTRIBUTION WITH RESPECT TO not-toxic



Eikös tämä Sammaljoki laske Kokemäenjokeen siinä Äetsä alapuolella, ja se Kokemäenjoki taas puolestaan laskee mereen?  " Turpaa esiintyy vain etelä- ja lounaisrannikon murtovesissä sekä niihin laskevissa joissa. ". Tämähän se taisi olla väitteeni?  En oletakaan, että tietäisit kaloista. Mutta miksi sinulle on niin tärkeää yrittää päteä asiassa, josta ei taida tietää mitään. Eihän ne faktat - veli hopea - jankuttamalla muuksi muutu. Eikä se muutu winkkarikaan, kusetusta mikä kusetusta.  Ymmärtäisit jo nousta siitä Sarviksen pulkasta ja olla kuin muutkin miehet, jooko.
ATTRIBUTION WITH RESPECT TO toxic



Prediction: not-toxic Weights: [3.044442653656006, -3.2175557613372803]
Etelän Vetelillä mennyt pupu pöksyyn ja peukalo keskelle kämmentä kun eivät saa itse lumiaan pois? Sinne vaan kaupunkilaiset lapio kouraan ja hihat heilumaan. Ei se ole kuin tahdosta kiinni. Pv:ltä vähän kalustoapua, niin eiköhän se siitä.
ATTRIBUTION WITH RESPECT TO not-toxic



Etelän Vetelillä mennyt pupu pöksyyn ja peukalo keskelle kämmentä kun eivät saa itse lumiaan pois? Sinne vaan kaupunkilaiset lapio kouraan ja hihat heilumaan. Ei se ole kuin tahdosta kiinni. Pv:ltä vähän kalustoapua, niin eiköhän se siitä.
ATTRIBUTION WITH RESPECT TO toxic


In [ ]:
for e in false_positive["text"][:-10]:
  predict_and_explain(model, e)

Prediction: toxic Weights: [0.31950506567955017, 0.3732818365097046]
kuitenkaan mikään feministi oo..
ATTRIBUTION WITH RESPECT TO not-toxic



kuitenkaan mikään feministi oo..
ATTRIBUTION WITH RESPECT TO toxic



Prediction: toxic Weights: [0.1805495172739029, 0.26252081990242004]
Asiasta: Blondit ja äly. Kaikki blondit eivät ole kauniita mielestäni. Vain älykkäät blondit ovat kauniita, äly näkyy silmissä. -Tyhmä "Gro magno" ei ole mielestäni koskaan kaunis.
ATTRIBUTION WITH RESPECT TO not-toxic



Asiasta: Blondit ja äly. Kaikki blondit eivät ole kauniita mielestäni. Vain älykkäät blondit ovat kauniita, äly näkyy silmissä. -Tyhmä "Gro magno" ei ole mielestäni koskaan kaunis.
ATTRIBUTION WITH RESPECT TO toxic



Prediction: toxic Weights: [-0.16203883290290833, 0.4198240041732788]
Ryppäämään, ryyppäämään, joka aamu sännätään...
ATTRIBUTION WITH RESPECT TO not-toxic


...


hakata sut miksi helevetissä? eihän tuossa oo mitää järkeä!pyytää sua tapaamiseen ja haluu hakata sut. oonkohan mä liian vanha ku en ymmärrä.
ATTRIBUTION WITH RESPECT TO toxic



Prediction: toxic Weights: [-3.6952452659606934, 4.1648383140563965]
Haluaisin syödä paskaasi.
ATTRIBUTION WITH RESPECT TO not-toxic



Haluaisin syödä paskaasi.
ATTRIBUTION WITH RESPECT TO toxic



Prediction: toxic Weights: [-3.7423312664031982, 4.121918201446533]
Sä et tu saamaan mitään. Miehet ja Kristukset aina vaan pienillä kirjaimilla. Mä otan sun karvaisen miehes, haen kadulta thainaisen ja käsken sen pesee nuollen yksitellen kaikki karvat siltä mieheltäs.
ATTRIBUTION WITH RESPECT TO not-toxic



Sä et tu saamaan mitään. Miehet ja Kristukset aina vaan pienillä kirjaimilla. Mä otan sun karvaisen miehes, haen kadulta thainaisen ja käsken sen pesee nuollen yksitellen kaikki karvat siltä mieheltäs.
ATTRIBUTION WITH RESPECT TO toxic
